# Ejemplo de Inferencia

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
! pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [9]:
import os
import shutil
import json
from ultralytics import YOLO
import cv2
import PIL.Image as Image
import imageio as iio

## Ajustes Iniciales

In [6]:
# Nombre del alumno
student_name = "simon_revello"

# Ruta al archivo de pesos
model_path = "/home/simon/Documentos/FCEIA/computer-vision/cartas_tp/runs/detect/train/weights/best.pt"

# Ruta al directorio que contiene las imagenes
imgs_dir = "/home/simon/Documentos/FCEIA/computer-vision/cartas_tp/dataset2/images/val"

# Ruta al directorio de destino de las detecciones
base_dir = "/home/simon/Documentos/FCEIA/computer-vision/cartas_tp/dataset2/out"
dets_dir = os.path.join(base_dir, student_name)

model = YOLO("/home/simon/Documentos/FCEIA/computer-vision/cartas_tp/runs/detect/train/weights/best.pt")

In [24]:
img = iio.imread("/home/simon/Documentos/FCEIA/computer-vision/cartas_tp/dataset2/images/train/F37605_florencia_fernandez_53.png")
results = model.predict(img, conf=0.3, imgsz=640, show_conf=True)
for r in results:
    print(len(r.boxes), r.probs, r.tojson())


0: 640x480 1 1C, 1 1E, 1 5C, 1 11C, 1 12E, 10.2ms
Speed: 19.6ms preprocess, 10.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)
5 None [
  {
    "name": "12E",
    "class": 46,
    "confidence": 0.985724151134491,
    "box": {
      "x1": 306.43365478515625,
      "y1": 792.3607177734375,
      "x2": 612.0839233398438,
      "y2": 1136.0711669921875
    }
  },
  {
    "name": "1E",
    "class": 2,
    "confidence": 0.9583532810211182,
    "box": {
      "x1": 595.8812255859375,
      "y1": 669.3236083984375,
      "x2": 932.0594482421875,
      "y2": 976.2109375
    }
  },
  {
    "name": "11C",
    "class": 41,
    "confidence": 0.9552983045578003,
    "box": {
      "x1": 250.10130310058594,
      "y1": 238.47430419921875,
      "x2": 482.594482421875,
      "y2": 556.7762451171875
    }
  },
  {
    "name": "1C",
    "class": 1,
    "confidence": 0.9438344240188599,
    "box": {
      "x1": 5.512786865234375,
      "y1": 572.8150634765625,
      "x2": 327.791717

/tmp/ipykernel_375108/312200581.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = iio.imread("/home/simon/Documentos/FCEIA/computer-vision/cartas_tp/dataset2/images/train/F37605_florencia_fernandez_53.png")


In [ ]:
# Reestablecimiento del directorio de destino (eliminacion)
if os.path.exists(dets_dir):
    shutil.rmtree(dets_dir)

os.makedirs(dets_dir)

## Inferencia

### Formato YOLOv5 para las salidas de las inferencias

Estas son las celdas para que el estudiante complete con su código de inferencia.

Debe generar un archivo con las detecciones en formato YOLOv5, donde cada línea contiene:
 cls, xc, yx, w, h, c

* **cls**: número de índice de la clase

* **xc**: coordenada x al centro de la caja delimitadora (bbox)

* **yx**: coordenada y al centro de la caja delimitadora (bbox)

* **w**: ancho de la caja delimitadora

* **h**: alto de la caja delimitadora

* **c**: certeza del modelo sobre la clasificacíon (confidence)


 Todas las coordenadas deben ser relativas: [0 ... 1]

NOTA: Si utiliza un modelo YoloV8 de ultralytics, pueder llamar al método result.save_txt(file_name, save_conf=True)

### Formato para los archivos de envido.json

Además de las detecciones anteriores, debe existir un archivo llamado envido.json con la siguiente estructura:

```json
{
    "IMG_20240630_174514649_HDR.jpg": {
        "total_cards": 3,
        "cards": {
            "E": [],
            "C": [
                12
            ],
            "B": [
                11
            ],
            "O": [
                10
            ]
        },
        "points": 0,
        "figure": "N/A"
    },
    "IMG_20240630_173918579.jpg": {
        "total_cards": 2,
        "cards": {
            "E": [],
            "C": [],
            "B": [],
            "O": [
                9,
                8
            ]
        },
        "points": 0,
        "figure": "N/A"
    },
  ...
}
```

Esto indica que hay un total de tres cartas, que son el 5 y el 1 de Copas, y el 2 de Basto, que dan 26 puntos de envido de Copa.

Este archivo debe contener una clave con el nombre de archivo, donde cada elemento es a su vez un objeto cons las claves detalladas.

Revise el archivo gt_envido.json para analizar todas las posibilidades.

### DEMO: cálculo del envido

A continuación hay una implementación del cálculo de los puntos a modo de ejemplo, que sirve de referencia para evaluar el algoritmo de evaluación.

**ESTA ES LA SECCIÓN QUE CADA ALUMNO REEMPLAZARIA CON SU CODIGO**

In [55]:
def get_txt_filename(img_path : str):
    img_filename = os.path.basename(img_path)
    return img_filename.replace(".jpg", ".txt").replace(".png", ".txt")

def read_card(img_path : str):
    cards = {"E": [], "C": [], "B": [], "O": [], "J": []}
    results = model.predict(img_path)
    resultado = {img_path: {}}
    for result in results:
        result.save_txt(os.path.join(dets_dir, get_txt_filename(result.path)), save_conf=True)
        arrayR = json.loads(result.tojson())
        total_cards = len(arrayR)
        resultado[img_path]["total_cards"] = total_cards
        for pred in arrayR:
            name = pred["name"]
            if name == "J":
                cards[name].append(1)
            else:
                if len(name) == 2:
                    clase = name[1]
                    nro = int(name[0])
                    cards[clase].append(nro)
                elif len(name) == 3:
                    clase = name[2]
                    nro = int(name[:2])
                    cards[clase].append(nro)
        resultado[img_path]["cards"] = cards
        resultado[img_path]["points"] = 0
        resultado[img_path]["figure"] = "N/A"

    if total_cards == 0: return resultado
    elif total_cards > 3:
        print(f"Sobra(n) {total_cards - 3} carta(s)")
        return resultado
    elif total_cards < 3:
        print(f"Falta(n) {3 - total_cards} carta(s)")
        return resultado
        
    hayCartaRara = False
    
    for tipoCarta in resultado[img_path]["cards"].keys():
        if tipoCarta == "J" and len(resultado[img_path]["cards"][tipoCarta]) > 0:
            hayCartaRara = True
            print("El comodín no es válido")
            break
        for nro in resultado[img_path]["cards"][tipoCarta]:
            if nro >= 8 and nro <= 12:
                hayCartaRara = True
                print(f"El nro {nro} no está habilitado para el truco")
                break
        if hayCartaRara: return resultado

    #Evaluar envido
    cantEspecial = 0
    cantIguales = 0
    figure = None
    totEnvido = 0
    for tipoCarta in resultado[img_path]["cards"].keys():
        if len(resultado[img_path]["cards"][tipoCarta]) <= 1: continue
        cantIguales = len(resultado[img_path]["cards"][tipoCarta])
        figure = tipoCarta
        for nro in resultado[img_path]["cards"][tipoCarta]:
            if nro > 9 and nro <= 12:
                cantEspecial += 1
                continue
            totEnvido += nro
    
    if cantIguales > 0:
        if cantIguales == cantEspecial:
            totEnvido = 20
        elif cantEspecial == 0:
            totEnvido += 20
    
    resultado[img_path]["figure"] = figure
    resultado[img_path]["points"] = totEnvido if totEnvido <= 33 else 33
    return resultado

In [56]:
print(read_card("/home/simon/Documentos/FCEIA/computer-vision/cartas_tp/IMG_20240728_130056.jpg"))


image 1/1 /home/simon/Documentos/FCEIA/computer-vision/cartas_tp/IMG_20240728_130056.jpg: 640x512 1 1E, 1 2O, 1 4E, 9.3ms
Speed: 1.9ms preprocess, 9.3ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 512)
{'/home/simon/Documentos/FCEIA/computer-vision/cartas_tp/IMG_20240728_130056.jpg': {'total_cards': 3, 'cards': {'E': [1, 4], 'C': [], 'B': [], 'O': [2], 'J': []}, 'points': 25, 'figure': 'E'}}


## Escritura del archivo envido.json

In [57]:
IMG_PATH = "/home/simon/Documentos/FCEIA/computer-vision/cartas_tp/IMG_20240728_130056.jpg"
with open(os.path.join(dets_dir, "envido.json"), "w") as jf:
    resultado = read_card(IMG_PATH)
    json.dump(resultado, jf, indent=4)


image 1/1 /home/simon/Documentos/FCEIA/computer-vision/cartas_tp/IMG_20240728_130056.jpg: 640x512 1 1E, 1 2O, 1 4E, 9.6ms
Speed: 10.3ms preprocess, 9.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 512)
